In [1]:
import numpy as np # linear algebra
#import random
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import random
import shutil
import tensorflow as tf# Enable TensorFlow 1.x behavior
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import shutil
from glob import glob
# Helper libraries
import matplotlib.pyplot as plt
#import math
%matplotlib inline
#print(tf.__version__)

AttributeError: module 'tensorflow._api.v2.compat.v1.__internal__' has no attribute 'dispatch'

In [ ]:
data_root='C:/Users/saich/Music/pneumonia/dataset'
path_positive_cases = os.path.join('C:/Users/saich/Music/pneumonia/dataset/pneumonia')
path_negative_cases = os.path.join('C:/Users/saich/Music/pneumonia/dataset/normal')

### Datasets Overview 

In [ ]:
# jpg and png files
positive_images_ls = glob(os.path.join(path_positive_cases,"*.jpeg"))

negative_images_ls = glob(os.path.join(path_negative_cases,"*.jpeg"))
negative_images_ls.extend(glob(os.path.join(path_negative_cases,"*.jpeg")))
len(positive_images_ls)

In [ ]:
pneumonia = {'class': 'pneumonia',
         'path': path_positive_cases,
         'images': positive_images_ls}

normal = {'class': 'normal',
             'path': path_negative_cases,
             'images': negative_images_ls}

In [ ]:
total_positive_pneumonia = len(positive_images_ls)
total_negative_pneumonia = len(negative_images_ls)
print("Total Positive pneumonia images: {}".format(total_positive_pneumonia))
print("Total Negative pneumonia images: {}".format(total_negative_pneumonia))

In [ ]:
image_pneumonia = cv2.imread(os.path.join(positive_images_ls[1]))
image_negative = cv2.imread(os.path.join(negative_images_ls[5]))

image1 = cv2.resize(image_pneumonia, (500, 600))
image2 = cv2.resize(image_negative, (500, 600))
# The initial processing of the image
# image = cv2.medianBlur(image, 3)
image_gr1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image_gr2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit = 5)
final_img1 = clahe.apply(image_gr1) + 30
final_img2 = clahe.apply(image_gr2) + 30
# Ordinary thresholding the same image
_, ordinary_img1 = cv2.threshold(image_gr1, 155, 255, cv2.THRESH_BINARY)
_, ordinary_img2= cv2.threshold(image_gr2, 155, 255, cv2.THRESH_BINARY)
# Showing all the three images
f = plt.figure(figsize=(8, 8))
f.add_subplot(3, 3, 1)

plt.imshow(image_negative)
f.add_subplot(3, 3, 2)
plt.imshow(image_pneumonia)
plt.title("pneumonia image")
f.add_subplot(3, 3, 3)
plt.imshow(image_gr1)
f.add_subplot(3, 3, 4)
plt.imshow(image_gr2)
f.add_subplot(3, 3, 5)
plt.imshow(final_img1)
f.add_subplot(3, 3, 6)
plt.imshow(final_img2)

In [ ]:
print("Image pneumonia Shape {}".format(image_pneumonia.shape))
print("Image normal Shape {}".format(image_negative.shape))

### Create Train-Test Directory 

In [ ]:
# Create Train-Test Directory
subdirs  = ['train/', 'test/']
for subdir in subdirs:
    labeldirs = ['pneumonia', 'normal']
    for labldir in labeldirs:
        newdir = subdir + labldir
        os.makedirs(newdir, exist_ok=True)
        print(newdir)

In [ ]:
# Copy Images to test set

# seed random number generator
random.seed(237)
# define ratio of pictures used for testing 
test_ratio = 0.2


for cases in [pneumonia, normal]:
    total_cases = len(cases['images']) #number of total images
    num_to_select = int(test_ratio * total_cases) #number of images to copy to test set
    
    print(cases['class'], num_to_select)
    
    list_of_random_files = random.sample(cases['images'], num_to_select) #random files selected

    for files in list_of_random_files:
        shutil.copy2(files, 'test/' + cases['class'])

In [ ]:
# Copy Images to train set
for cases in [pneumonia, normal]:
    image_test_files = os.listdir('test/' + cases['class']) # list test files 
    for images in cases['images']:
        if images.split('/')[-1] not in (image_test_files): #exclude test files from shutil.copy
            shutil.copy2(images, 'train/' + cases['class'])

In [ ]:
total_train_pneumonia = len(os.listdir('C:/Users/saich/Music/pneumonia/train/pneumonia'))
total_train_normal = len(os.listdir('C:/Users/saich/Music/pneumonia/train/normal'))
total_test_pneumonia = len(os.listdir('C:/Users/saich/Music/pneumonia/test/pneumonia'))
total_test_normal = len(os.listdir('C:/Users/saich/Music/pneumonia/test/pneumonia/normal'))

print("Train sets images pneumonia: {}".format(total_train_pneumonia))
print("Train sets images normal: {}".format(total_train_normal))
print("Test sets images pneumonia: {}".format(total_test_pneumonia))
print("Test sets images normal: {}".format(total_test_normal))

### Simple CNN Model
[Tensorflow Tutorial](https://www.tensorflow.org/tutorials/images/classification)

In [ ]:
batch_size =55
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
test_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [ ]:
train_dir = os.path.join('C:/Users/saich/Music/pneumonia/dataset')
test_dir = os.path.join('C:/Users/saich/Music/pneumonia/dataset')


total_train = total_train_pneumonia + total_train_normal
total_test = total_test_pneumonia + total_test_normal

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [ ]:
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=test_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [ ]:
model = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(2, 2),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
        
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=test_data_gen,
    validation_steps=total_test // batch_size
)

In [ ]:
history = model.history.history

In [ ]:
def plot_metrics(history):
    
    train_loss = history['loss']
    val_loss = history['val_loss']
    train_acc = history['accuracy']
    val_acc = history['val_accuracy']
    
    plt.figure()

    # Loss
    

    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    plt.show()
    
    # Accuracy
   
    plt.figure()
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()
 
    plt.show()


In [ ]:
plot_metrics(history)

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from keras.preprocessing import image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image
#/content/drive/MyDrive/Lakshmi Meghana/brain tumor python/BRAIN PYTHON/test/normal/1.jpg
img=mpimg.imread('C:/Users/saich/Music/pneumonia/test/normal/IM-0262-0001.jpeg')
imgplot = plt.imshow(img, interpolation='none')


test_image = image.load_img('C:/Users/saich/Music/pneumonia/dataset/pneumonia/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-000-fig1b.png', target_size = (150, 150))
#C:\Users\HP\Desktop\pneumonia\dataset\normal

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)
if result==1:
    print( 'normal')
else:
    print( 'pneumonia')


    

In [ ]:
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

img=mpimg.imread('C:/Users/saich/Music/pneumonia/test/normal/IM-0262-0001.jpeg')
imgplot = plt.imshow(img, interpolation='none')

test_image = image.load_img('C:/Users/saich/Music/pneumonia/test/normal/IM-0262-0001.jpeg',target_size = (150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)
if result[0]>=1:
    prediction = 'normal'
else:
    prediction = 'pneumonia'

plt=plt.title('Prediction is  '+ prediction )
